In [1]:
import os
import librosa, librosa.display
import matplotlib.pyplot as plt
import math

In [ ]:
for root, folders, files in os.walk('/content/drive/MyDrive/Python Project/sound'):
        # going into sound folder in drive
    for folder in folders:
        os.makedirs(f'/content/img_data/{folder}',0o666)     #creating folder
        print(f'Processing {folder}')   #prefix f is for formatting
        for _root, _folders, _files in os.walk(f'/content/drive/MyDrive/Python Project/sound/{folder}'):
                #going into every folders of sound
            for file in _files:
                  #going into every files of the folders in sound
                signal, sr = librosa.load(f'/content/drive/MyDrive/Python Project/sound/{folder}/{file}', sr=22050)
                size = signal.shape[0]
                start = 0
                end = start + (size//5) #dividing the sound into 5 parts for more accuracy
                for i in range(5):
                    part_of_signal = signal[start:end]
                    mfcc = librosa.feature.mfcc(part_of_signal,
                                n_fft=2048,
                                hop_length=512,
                                n_mfcc=13)
                    librosa.display.specshow(mfcc, sr=sr, hop_length=512)
                    plt.savefig(f'/content/img_data/{folder}/{file[:-4] +"_"+ str(i)}.png')
                    start = end
                    end = start + (size//5)

In [3]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
training_set = ImageDataGenerator(rescale=1./255)
test_set = ImageDataGenerator(rescale=1./255)

In [5]:
! pip install split-folders

In [6]:
import splitfolders
splitfolders.ratio('img_data', output="out", seed=1337, ratio=(.9, .1))

Copying files: 505 files [00:00, 9225.56 files/s]


In [7]:
training_data = training_set.flow_from_directory(
        '/content/out/train',
        batch_size=1,
        target_size=(128, 128))

val_data = test_set.flow_from_directory(
        '/content/out/val',
        batch_size=1,
        target_size=(128, 128))

Found 453 images belonging to 5 classes.
Found 52 images belonging to 5 classes.
